# HRTEM analyse

In [3]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
import hyperspy.api as hs
import pyxem as pxm

from matplotlib.widgets import Slider
from skimage import exposure

from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import matplotlib.patheffects as patheffects

In [4]:
s = hs.load("Noe_HRETEM_greier.dm3")

In [3]:
s.plot()

In [4]:
s.axes_manager

Signal axis name,size,,offset,scale,units
x,1024,,-0.0,0.013701171614229679,nm
y,1024,,-0.0,0.013701171614229679,nm


## Set limits to enhance contrast

Interactive to find values

In [27]:
# Load the data (assumes `s` is already loaded and contains the image data)
image_data = s.data

# Initial percentiles for contrast stretching
initial_p2, initial_p98 = 0.5, 99

# Create the figure and initial plot
fig, ax = plt.subplots(figsize=(8, 8))
plt.subplots_adjust(bottom=0.25)  # Make room for sliders

# Apply initial contrast stretching
p2, p98 = np.percentile(image_data, (initial_p2, initial_p98))
stretched_data = exposure.rescale_intensity(image_data, in_range=(p2, p98))
im = ax.imshow(stretched_data, cmap='gray', origin='upper')
ax.axis('off')

# Create sliders for adjusting the percentiles
ax_p2 = plt.axes([0.2, 0.1, 0.65, 0.03], facecolor='lightgray')
ax_p98 = plt.axes([0.2, 0.05, 0.65, 0.03], facecolor='lightgray')

slider_p2 = Slider(ax_p2, 'P2', 0, 10, valinit=initial_p2, valstep=0.1)
slider_p98 = Slider(ax_p98, 'P98', 90, 100, valinit=initial_p98, valstep=0.1)

# Update function to apply contrast stretching based on slider values
def update(val):
    p2 = np.percentile(image_data, slider_p2.val)
    p98 = np.percentile(image_data, slider_p98.val)
    stretched_data = exposure.rescale_intensity(image_data, in_range=(p2, p98))
    im.set_data(stretched_data)
    fig.canvas.draw_idle()

# Connect the sliders to the update function
slider_p2.on_changed(update)
slider_p98.on_changed(update)

# Show the interactive plot
plt.show()


Then use suitable values for p1 and p2

In [32]:
# Apply contrast stretching
p1, p2 = np.percentile(s.data, (0.5, 99.9))  # Clip the extreme p1% and p2%
s_stretch = s.deepcopy()
s_stretch.data = exposure.rescale_intensity(s.data, in_range=(p1, p2))
s_stretch.plot()


In [ ]:
s.axes_manager.signal_extent # når vi bruker imshow(extent=...) setter vi altså 0 til 14.06 nm for x-aksen, 0 til 14.06 for y-aksen.

(0.0, 14.016298561356962, 0.0, 14.016298561356962)

## Make figure and save plot

In [35]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(s_stretch.data, extent = s.axes_manager.signal_extent, cmap='gray')
ax.axis('off')  # Remove axes for clarity

fontprops = fm.FontProperties(size=25)
scalebar = AnchoredSizeBar(
    transform=ax.transData,
    size=3,
    label="3 nm",
    loc=4,
    frameon=False,
    color="white",
    size_vertical=0.25,
    label_top=False,
    fontproperties=fontprops)
scalebar.txt_label._text.set_path_effects([patheffects.withStroke(linewidth=2, foreground='black', capstyle="round")])
ax.add_artist(scalebar)

fig.savefig("HRTEM.png", bbox_inches='tight', pad_inches=0, dpi=300)